My notes for background to problem -- implementing rate limiting into my use of the Evernote API ( http://dev.evernote.com/doc/articles/rate_limits.php )

https://www.evernote.com/shard/s1/sh/5327734c-fdfd-4d1c-9af4-8a32ad7a2e51/2d234b29342f3b8c8c4ec3d180c8861d

Basic approach to explore:

* proxying the existing EvernoteClient object or client
* learning from Python decorators about how to do a retry
* down the road, look at using `APScheduler` or `celery` to do more sophisticated handling of asynchronous retries.

I was considering using a class decorator: http://my.safaribooksonline.com/book/programming/python/9780768687040/classes-and-object-oriented-programming/ch07lev1sec17

In [ ]:
import evernote

from time import sleep
from evernote.edam.error.ttypes import (EDAMSystemException, EDAMErrorCode)

def evernote_rate_limit(f):
    def f2(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except EDAMSystemException, e:
            if e.errorCode == EDAMErrorCode.RATE_LIMIT_REACHED:
                sleep(e.rateLimitDuration)
                return f(*args, **kwargs)
    
    return f2


class RateLimitingEvernoteProxy(object):
    __slots__ = ["_obj"]
    def __init__(self, obj):
        object.__setattr__(self, "_obj", obj)
    
    def __getattribute__(self, name):
        return evernote_rate_limit(getattr(object.__getattribute__(self, "_obj"), name))

In [ ]:
# settings holds the devToken/authToken that can be used to access Evernote account
#http://dev.evernote.com/doc/articles/authentication.php#devtoken
# settings.authToken

import settings


from evernote.api.client import EvernoteClient

dev_token = settings.authToken

client = RateLimitingEvernoteProxy(EvernoteClient(token=dev_token, sandbox=False))

userStore = client.get_user_store()
user = userStore.getUser()
print user.username

In [ ]:
type(client)